# Import Resources

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import numpy as np
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import pymysql
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


# **DATA BASE**

In [2]:
## VARIABLE DEFINITIONS

ssh_ip_address = ''
ssh_p_username = ''
ssh_p_password = ""
ssh_port = 22
ssh_remote_bind_address = 3306
db_server_ip = ''
db_server_port = 3306
db_user = ''
db_user_password = ''
db_name = ''

## SSH CONNECTION

tunnel = SSHTunnelForwarder((ssh_ip_address, 22), ssh_password=ssh_p_password, ssh_username=ssh_p_username,
                            remote_bind_address=('127.0.0.1', 3306))
tunnel.start()
# Establish an SSH tunnel to the remote server using the specified IP address, port, username, and password
# The tunnel will bind to the local address '127.0.0.1' and port 3306

# MYSQL CONNECTION
db = pymysql.connect(host='localhost', port=tunnel.local_bind_port, user=ssh_p_username,
                          passwd=db_user_password, db=db_name)
# Establish a connection to the MySQL server using the local bind port of the SSH tunnel, username, password, and database name


# Create a cursor object to execute SQL queries on the database
cur = db.cursor()

In [3]:
# SQL query to select all data from the table
query = "SELECT * FROM TI_PV_main;"
# Execute the query
cur.execute(query)
# Fetch all the rows
rows1 = cur.fetchall()

# SQL query to select all data from the table
query = "SELECT * FROM TI_PV_branch;"
# Execute the query
cur.execute(query)
# Fetch all the rows
rows2 = cur.fetchall()


df1 = pd.DataFrame(rows1)
df2 = pd.DataFrame(rows2)

# Drop two columns from the first DataFrame
columns_to_drop = [0, 1]
df2 = df2.drop(columns=columns_to_drop)

# # Concatenate the DataFrames
df_c = pd.concat([df1, df2], axis=1)
Labels=df_c[df_c.columns[1]].tolist()

X= df_c.drop(df_c.columns[:2], axis=1)

# Modify labels
for i in range(0,len(Labels)):
    if Labels[i] =='No-Anomaly':
        pass
    else:
        Labels[i]='Anomaly'


# Create an instance of the MinMaxScaler
scaler = MinMaxScaler()
# Normalize the data in the DataFrame
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
y=Labels



In [4]:
y=Labels

# Prepared data for training

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=55)

In [46]:
# To select the top 20 most important features using mutual_info_classif:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=300) 
X_selected = selector.fit_transform(X, y)
#X_selected=X

# SIMPLE DECISION TREE

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# # Define the hyperparameters to tune
# parameters = {
#     'max_depth': [2, 4],
#     'min_samples_split': [2],
#     'min_samples_leaf': [1]
# }

# # Create the Decision Tree model
# model = DecisionTreeClassifier()

# # Create the GridSearchCV object
# grid_search = GridSearchCV(estimator=model, 
#                            param_grid=parameters, 
#                            refit='r2', 
#                            cv=2,
#                            #verbose=4 # how much information we wannna print
# )

# # Perform Grid Search
# grid_search.fit(X, y)

# # Get the best parameters and the best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# # Print the results
# print("Best parameters:", best_params)
# print("Best score:", best_score)

Save results of Grid Search in a .csv file

In [15]:
# df= pd.DataFrame(grid_search.cv_results_)
# df.to_csv("GS_results_DecisionTreeSimple")

Save the best model

In [9]:
import pickle

# # Train the model and get the best model with GridSearchCV
# best_model = grid_search.best_estimator_

# # Save the model to a file
# model_file = 'best_model_DecisionTreeeSimple.pkl'
# with open(model_file, 'wb') as f:
#     pickle.dump(best_model, f)



## Bagging

In [ ]:
# from sklearn.ensemble import BaggingClassifier


# # Define the hyperparameters to tune
# parameters = {
#     'base_estimator__max_depth': [2, 4, 6, 8],
#     'base_estimator__min_samples_split': [2, 4, 6, 8],
#     'base_estimator__min_samples_leaf': [1, 2, 3, 4],
#     'n_estimators': [10, 50, 100],
#     'max_samples': [0.5, 0.8, 1.0],
#     'max_features': [0.5, 0.8, 1.0]
# }

# # Create the base estimator (Decision Tree)
# base_estimator = DecisionTreeClassifier()

# # Create the BaggingClassifier model
# model = BaggingClassifier(base_estimator=base_estimator)

# # Create the GridSearchCV object
# grid_search = GridSearchCV(estimator=model,
#                            param_grid=parameters,
#                            cv=5)

# # Perform Grid Search
# grid_search.fit(X, y)

# # Get the best parameters and the best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# # Print the results
# print("Best parameters:", best_params)
# print("Best score:", best_score)

## Random Forest

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# # Define the hyperparameters to tune
# parameters = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [2, 5, 10],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Create the Random Forest model
# model = RandomForestClassifier()

# # Create the GridSearchCV object
# grid_search = GridSearchCV(estimator=model, param_grid=parameters, cv=5)

# # Perform Grid Search
# grid_search.fit(X, y)

# # Get the best parameters and the best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# # Print the results
# print("Best parameters:", best_params)
# print("Best score:", best_score)

## Gradient Boosting

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier

# # Define the hyperparameters to tune
# parameters = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 5, 7],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4]
# }

# # Create the Gradient Boosting model
# model = GradientBoostingClassifier()

# # Create the GridSearchCV object
# grid_search = GridSearchCV(estimator=model, param_grid=parameters, cv=5)

# # Perform Grid Search
# grid_search.fit(X, y)

# # Get the best parameters and the best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# # Print the results
# print("Best parameters:", best_params)
# print("Best score:", best_score)